### Neural Networks for User schedule based Playlist generation
#### Author: Rhema Ike

In [243]:
# Imports
import pandas as pd
import numpy as np
import os
import time

from sklearn import preprocessing as preproc



In [292]:
#Loading Data set for 

DATA_DIR = "DataSets"
file_name = "SongDataSetLess_4000.xlsx"
data_file_name = os.path.join(DATA_DIR, file_name)

df = pd.read_excel(data_file_name)
df.head()

,name,artist,key,liveness,instrumentalness,loudness,mode,speechiness,tempo,valence,danceability,energy,acousticness
0,Losing It,FISHER,2,0.0974,0.700000,-5.844,1,0.0576,125.000,0.6410,0.760,0.964,0.00182
1,Midnight Hour with Boys Noize & Ty Dolla $ign,Skrillex,0,0.1990,0.000392,-4.202,1,0.0685,125.978,0.3290,0.795,0.899,0.00235
2,Clouds,Zonderling,1,0.7780,0.840000,-5.362,1,0.0957,125.953,0.0813,0.681,0.926,0.04950
3,Hypnagogic (I Can't Wait) [edit],Love Regenerator,9,0.1080,0.766000,-3.852,0,0.0458,139.982,0.2500,0.422,0.957,0.00288
4,Your Mind - Radio Edit,Adam Beyer,11,0.2150,0.165000,-7.946,0,0.0565,126.032,0.1110,0.660,0.857,0.00262


In [293]:
df_r,df_c = df.shape
PLAYLIST_SIZE = 1000
tempodfs = []
prev_r = 0
for r in range(PLAYLIST_SIZE, df_r+PLAYLIST_SIZE, PLAYLIST_SIZE):
    tempodfs.append(pd.DataFrame(df["tempo"].iloc[prev_r:r]))
    prev_r = r
#     print(r)
    
tempodfs[0].head()

for i,tempodf in enumerate(tempodfs):
    new_df = preproc.scale(tempodf["tempo"])
    tempodfs[i] = pd.DataFrame(new_df)

    
# new_df = pd.DataFrame(columns=["tempo"])
new_df = pd.concat(tempodfs)
new_df = new_df.rename(columns={0: "normtemp"}).reset_index(drop=True)
new_df.count()
df.count()

df = pd.concat([df,new_df], axis=1)
df.head()

,name,artist,key,liveness,instrumentalness,loudness,mode,speechiness,tempo,valence,danceability,energy,acousticness,normtemp
0,Losing It,FISHER,2,0.0974,0.700000,-5.844,1,0.0576,125.000,0.6410,0.760,0.964,0.00182,-0.137419
1,Midnight Hour with Boys Noize & Ty Dolla $ign,Skrillex,0,0.1990,0.000392,-4.202,1,0.0685,125.978,0.3290,0.795,0.899,0.00235,-0.099438
2,Clouds,Zonderling,1,0.7780,0.840000,-5.362,1,0.0957,125.953,0.0813,0.681,0.926,0.04950,-0.100409
3,Hypnagogic (I Can't Wait) [edit],Love Regenerator,9,0.1080,0.766000,-3.852,0,0.0458,139.982,0.2500,0.422,0.957,0.00288,0.444409
4,Your Mind - Radio Edit,Adam Beyer,11,0.2150,0.165000,-7.946,0,0.0565,126.032,0.1110,0.660,0.857,0.00262,-0.097341


In [294]:
# Data Preprocessing Functions

# Normalize the data frame
def normalize(df, drop=[], hot_key = {}):
    
    df_r,df_ = df.shape
    
    df.drop(drop,axis=1,inplace=True)
    for col in df.columns:
        if col in ["loudness", "tempo"]:
            df[col+"norm"] = preproc.scale(df[col])
            
    for hot_key_col, vec_size in hot_key.items():
        ls = [] * vec_size
        
        # Create new dic tionarry with new column name and all key values 
        # set to a list of zeroes
        hot_key_dic = {}
        for r in range(vec_size):
            name = hot_key_col + str(r+1)
            hot_key_dic[name] = [0] * df_r
            
        hot_key_df = pd.DataFrame(hot_key_dic) 
        for i in range(df_r):
            hot_key_vec = [0] * vec_size
            index = int(df[hot_key_col].iloc[i])
            hot_key_vec[index] = 1
            hot_key_df.iloc[i] = hot_key_vec
        
        df = pd.concat([df, hot_key_df], axis=1)
            
    return df

In [295]:
df = normalize(df, drop = [], hot_key={"key": 12})
df.head(4)
df.columns

Index(['name', 'artist', 'key', 'liveness', 'instrumentalness', 'loudness',
       'mode', 'speechiness', 'tempo', 'valence', 'danceability', 'energy',
       'acousticness', 'normtemp', 'loudnessnorm', 'temponorm', 'key1', 'key2',
       'key3', 'key4', 'key5', 'key6', 'key7', 'key8', 'key9', 'key10',
       'key11', 'key12'],
      dtype='object')

In [296]:
new_df = df.iloc[:1000]
new_df.head()

,name,artist,key,liveness,instrumentalness,loudness,mode,speechiness,tempo,valence,...,key3,key4,key5,key6,key7,key8,key9,key10,key11,key12
0,Losing It,FISHER,2,0.0974,0.700000,-5.844,1,0.0576,125.000,0.6410,...,1,0,0,0,0,0,0,0,0,0
1,Midnight Hour with Boys Noize & Ty Dolla $ign,Skrillex,0,0.1990,0.000392,-4.202,1,0.0685,125.978,0.3290,...,0,0,0,0,0,0,0,0,0,0
2,Clouds,Zonderling,1,0.7780,0.840000,-5.362,1,0.0957,125.953,0.0813,...,0,0,0,0,0,0,0,0,0,0
3,Hypnagogic (I Can't Wait) [edit],Love Regenerator,9,0.1080,0.766000,-3.852,0,0.0458,139.982,0.2500,...,0,0,0,0,0,0,0,1,0,0
4,Your Mind - Radio Edit,Adam Beyer,11,0.2150,0.165000,-7.946,0,0.0565,126.032,0.1110,...,0,0,0,0,0,0,0,0,0,1


In [297]:
key1_columns = []
key2_columns = []
key_columns = []
num_keys = 12
for r in range(num_keys):
    key1_name = "key1" + str(r+1)
    key2_name = "key2" + str(r+1)
    key_name = "key" + str(r+1)
    key1_columns.append(key1_name)
    key2_columns.append(key2_name)
    key_columns.append(key_name)
    
key_columns    

['key1',
 'key2',
 'key3',
 'key4',
 'key5',
 'key6',
 'key7',
 'key8',
 'key9',
 'key10',
 'key11',
 'key12']

In [298]:
df[key_columns].iloc[0]
mix_df.columns

Index(['name1', 'artist1', 'key1', 'mode1', 'tempo1', 'name2', 'artist2',
       'key2', 'mode2', 'tempo2', 'y', 'key11', 'key12', 'key13', 'key14',
       'key15', 'key16', 'key17', 'key18', 'key19', 'key110', 'key111',
       'key112', 'key21', 'key22', 'key23', 'key24', 'key25', 'key26', 'key27',
       'key28', 'key29', 'key210', 'key211', 'key212'],
      dtype='object')

### Making Data for Energy Boost RNN and targets as well

In [281]:
DATA_SET_LENGTH = 250000


group_length = int(pow(DATA_SET_LENGTH, 0.5))
df = df.iloc[:group_length]
rng = DATA_SET_LENGTH

y_list = [0] * rng
y_df = pd.DataFrame(y_list, columns=["y"], dtype="int32")

data = [0] * rng
str_data = [""]*rng
x_dic = {"key1": data, "mode1": data, "tempo1": data, "key2": data, "mode2": data, "tempo2": data}
mix_dic = {"name1":str_data, "artist1":str_data, "key1":data, "mode1": data, "tempo1": data, "name2":str_data, "artist2":str_data, "key2": data, "mode2": data, "tempo2": data, "y":y_list}
cols = ["name1","artist1","key1","mode1","tempo1","name2","artist2","key2","mode2","tempo2","y"] + key1_columns+ key2_columns
mix_df = pd.DataFrame(mix_dic, columns=cols)
x_df = pd.DataFrame(x_dic,columns=["key1","mode1","tempo1","key2","mode2","tempo2"])

max_song_key = 11
min_song_key = 0
row_count = 0

tempo_lim = 0.04  # 4 percent
for _, row in df.iterrows():
    song1_key = int(row["key"])
    song1_mode = int(row["mode"])
    song1_tempo = (row["tempo"])
    song1_artist = row["artist"]
    song1_name = row["name"]
    tempo_high_bound = song1_tempo + (song1_tempo * tempo_lim)
    tempo_low_bound = song1_tempo - (song1_tempo * tempo_lim/2)
    for _, row2 in df.iterrows():
        song2_key = int(row2["key"])
        song2_mode = int(row2["mode"])
        song2_tempo = (row2["tempo"])
        song2_name = row2["name"]
        song2_artist = row2["artist"]

        # Inputing data in new data set
        mix_df.loc[row_count,"key1"] = song1_key
        mix_df.loc[row_count,"mode1"] = song1_mode
        mix_df.loc[row_count,"tempo1"] = song1_tempo
        mix_df.loc[row_count,"name1"] = song1_name
        mix_df.loc[row_count,"artist1"] = song1_artist
        mix_df.loc[row_count,"key2"] = song2_key
        mix_df.loc[row_count,"mode2"] = song2_mode
        mix_df.loc[row_count,"tempo2"] = song2_tempo
        mix_df.loc[row_count,"name2"] = song2_name
        mix_df.loc[row_count,"artist2"] = song2_artist
        
        mix_df.loc[row_count,key1_columns] = row[key_columns].values
        mix_df.loc[row_count,key2_columns] = row2[key_columns].values
        
#         for col in key1_columns:
#             if song1_key == int(col[3:]):
#                 mix_df[col][row_count] = 1 
#             else:
#                 mix_df[col][row_count] = 0
#         for col in key2_columns:
#             if song2_key == int(col[3:]):
#                 mix_df[col][row_count] = 1 
#             else:
#                 mix_df[col][row_count] = 0

        if song2_tempo > tempo_low_bound and song2_tempo < tempo_high_bound:
            if song1_mode == 0: # Key is Minor
                if song1_key == song2_key and song1_mode != song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song2_key == (song1_key + 1) and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song1_key == max_song_key and song2_key == min_song_key and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                else:
                    mix_df["y"][row_count] = 0
            if song1_mode == 1: # Key is Major
                if song2_key == (song1_key + 1) and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song1_key == max_song_key and song2_key == min_song_key and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                else:
                    mix_df["y"][row_count] = 0
        else:
            mix_df["y"][row_count] = 0 

        
        if(row_count % 100 == 0):
            prog = ((row_count + 1)/ rng) * 100
            print("### progress: {:.2f} % ({}/{})".format(prog, row_count+1, rng))
        row_count += 1

C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### progress: 0.00 % (1/250000)


C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### progress: 0.04 % (101/250000)
### progress: 0.08 % (201/250000)
### progress: 0.12 % (301/250000)
### progress: 0.16 % (401/250000)
### progress: 0.20 % (501/250000)
### progress: 0.24 % (601/250000)
### progress: 0.28 % (701/250000)
### progress: 0.32 % (801/250000)
### progress: 0.36 % (901/250000)
### progress: 0.40 % (1001/250000)
### progress: 0.44 % (1101/250000)
### progress: 0.48 % (1201/250000)
### progress: 0.52 % (1301/250000)
### progress: 0.56 % (1401/250000)
### progress: 0.60 % (1501/250000)


C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

### progress: 60.04 % (150101/250000)
### progress: 60.08 % (150201/250000)
### progress: 60.12 % (150301/250000)
### progress: 60.16 % (150401/250000)
### progress: 60.20 % (150501/250000)
### progress: 60.24 % (150601/250000)
### progress: 60.28 % (150701/250000)
### progress: 60.32 % (150801/250000)
### progress: 60.36 % (150901/250000)
### progress: 60.40 % (151001/250000)
### progress: 60.44 % (151101/250000)
### progress: 60.48 % (151201/250000)
### progress: 60.52 % (151301/250000)
### progress: 60.56 % (151401/250000)
### progress: 60.60 % (151501/250000)
### progress: 60.64 % (151601/250000)
### progress: 60.68 % (151701/250000)
### progress: 60.72 % (151801/250000)
### progress: 60.76 % (151901/250000)
### progress: 60.80 % (152001/250000)
### progress: 60.84 % (152101/250000)
### progress: 60.88 % (152201/250000)
### progress: 60.92 % (152301/250000)
### progress: 60.96 % (152401/250000)
### progress: 61.00 % (152501/250000)
### progress: 61.04 % (152601/250000)
### progress

### progress: 68.68 % (171701/250000)
### progress: 68.72 % (171801/250000)
### progress: 68.76 % (171901/250000)
### progress: 68.80 % (172001/250000)
### progress: 68.84 % (172101/250000)
### progress: 68.88 % (172201/250000)
### progress: 68.92 % (172301/250000)
### progress: 68.96 % (172401/250000)
### progress: 69.00 % (172501/250000)
### progress: 69.04 % (172601/250000)
### progress: 69.08 % (172701/250000)
### progress: 69.12 % (172801/250000)
### progress: 69.16 % (172901/250000)
### progress: 69.20 % (173001/250000)
### progress: 69.24 % (173101/250000)
### progress: 69.28 % (173201/250000)
### progress: 69.32 % (173301/250000)
### progress: 69.36 % (173401/250000)
### progress: 69.40 % (173501/250000)
### progress: 69.44 % (173601/250000)
### progress: 69.48 % (173701/250000)
### progress: 69.52 % (173801/250000)
### progress: 69.56 % (173901/250000)
### progress: 69.60 % (174001/250000)
### progress: 69.64 % (174101/250000)
### progress: 69.68 % (174201/250000)
### progress

### progress: 77.32 % (193301/250000)
### progress: 77.36 % (193401/250000)
### progress: 77.40 % (193501/250000)
### progress: 77.44 % (193601/250000)
### progress: 77.48 % (193701/250000)
### progress: 77.52 % (193801/250000)
### progress: 77.56 % (193901/250000)
### progress: 77.60 % (194001/250000)
### progress: 77.64 % (194101/250000)
### progress: 77.68 % (194201/250000)
### progress: 77.72 % (194301/250000)
### progress: 77.76 % (194401/250000)
### progress: 77.80 % (194501/250000)
### progress: 77.84 % (194601/250000)
### progress: 77.88 % (194701/250000)
### progress: 77.92 % (194801/250000)
### progress: 77.96 % (194901/250000)
### progress: 78.00 % (195001/250000)
### progress: 78.04 % (195101/250000)
### progress: 78.08 % (195201/250000)
### progress: 78.12 % (195301/250000)
### progress: 78.16 % (195401/250000)
### progress: 78.20 % (195501/250000)
### progress: 78.24 % (195601/250000)
### progress: 78.28 % (195701/250000)
### progress: 78.32 % (195801/250000)
### progress

### progress: 85.96 % (214901/250000)
### progress: 86.00 % (215001/250000)
### progress: 86.04 % (215101/250000)
### progress: 86.08 % (215201/250000)
### progress: 86.12 % (215301/250000)
### progress: 86.16 % (215401/250000)
### progress: 86.20 % (215501/250000)
### progress: 86.24 % (215601/250000)
### progress: 86.28 % (215701/250000)
### progress: 86.32 % (215801/250000)
### progress: 86.36 % (215901/250000)
### progress: 86.40 % (216001/250000)
### progress: 86.44 % (216101/250000)
### progress: 86.48 % (216201/250000)
### progress: 86.52 % (216301/250000)
### progress: 86.56 % (216401/250000)
### progress: 86.60 % (216501/250000)
### progress: 86.64 % (216601/250000)
### progress: 86.68 % (216701/250000)
### progress: 86.72 % (216801/250000)
### progress: 86.76 % (216901/250000)
### progress: 86.80 % (217001/250000)
### progress: 86.84 % (217101/250000)
### progress: 86.88 % (217201/250000)
### progress: 86.92 % (217301/250000)
### progress: 86.96 % (217401/250000)
### progress

### progress: 94.60 % (236501/250000)
### progress: 94.64 % (236601/250000)
### progress: 94.68 % (236701/250000)
### progress: 94.72 % (236801/250000)
### progress: 94.76 % (236901/250000)
### progress: 94.80 % (237001/250000)
### progress: 94.84 % (237101/250000)
### progress: 94.88 % (237201/250000)
### progress: 94.92 % (237301/250000)
### progress: 94.96 % (237401/250000)
### progress: 95.00 % (237501/250000)
### progress: 95.04 % (237601/250000)
### progress: 95.08 % (237701/250000)
### progress: 95.12 % (237801/250000)
### progress: 95.16 % (237901/250000)
### progress: 95.20 % (238001/250000)
### progress: 95.24 % (238101/250000)
### progress: 95.28 % (238201/250000)
### progress: 95.32 % (238301/250000)
### progress: 95.36 % (238401/250000)
### progress: 95.40 % (238501/250000)
### progress: 95.44 % (238601/250000)
### progress: 95.48 % (238701/250000)
### progress: 95.52 % (238801/250000)
### progress: 95.56 % (238901/250000)
### progress: 95.60 % (239001/250000)
### progress

### Making Data for Energy Drop RNN and targets as well

In [ ]:
DATA_SET_LENGTH = 250000


group_length = int(pow(DATA_SET_LENGTH, 0.5))
df = df.iloc[:group_length]
rng = DATA_SET_LENGTH

y_list = [0] * rng
y_df = pd.DataFrame(y_list, columns=["y"], dtype="int32")

data = [0] * rng
str_data = [""]*rng
x_dic = {"key1": data, "mode1": data, "tempo1": data, "key2": data, "mode2": data, "tempo2": data}
mix_dic = {"name1":str_data, "artist1":str_data, "key1":data, "mode1": data, "tempo1": data, "name2":str_data, "artist2":str_data, "key2": data, "mode2": data, "tempo2": data, "y":y_list}
cols = ["name1","artist1","key1","mode1","tempo1","name2","artist2","key2","mode2","tempo2","y"] + key1_columns+ key2_columns
mix_df = pd.DataFrame(mix_dic, columns=cols)
x_df = pd.DataFrame(x_dic,columns=["key1","mode1","tempo1","key2","mode2","tempo2"])

max_song_key = 11
min_song_key = 0
row_count = 0

tempo_lim = 0.04  # 4 percent
for _, row in df.iterrows():
    song1_key = int(row["key"])
    song1_mode = int(row["mode"])
    song1_tempo = (row["tempo"])
    song1_artist = row["artist"]
    song1_name = row["name"]
    tempo_high_bound = song1_tempo + (song1_tempo * tempo_lim)
    tempo_low_bound = song1_tempo - (song1_tempo * tempo_lim/2)
    for _, row2 in df.iterrows():
        song2_key = int(row2["key"])
        song2_mode = int(row2["mode"])
        song2_tempo = (row2["tempo"])
        song2_name = row2["name"]
        song2_artist = row2["artist"]

        # Inputing data in new data set
        mix_df.loc[row_count,"key1"] = song1_key
        mix_df.loc[row_count,"mode1"] = song1_mode
        mix_df.loc[row_count,"tempo1"] = song1_tempo
        mix_df.loc[row_count,"name1"] = song1_name
        mix_df.loc[row_count,"artist1"] = song1_artist
        mix_df.loc[row_count,"key2"] = song2_key
        mix_df.loc[row_count,"mode2"] = song2_mode
        mix_df.loc[row_count,"tempo2"] = song2_tempo
        mix_df.loc[row_count,"name2"] = song2_name
        mix_df.loc[row_count,"artist2"] = song2_artist
        
        mix_df.loc[row_count,key1_columns] = row[key_columns].values
        mix_df.loc[row_count,key2_columns] = row2[key_columns].values
        
#         for col in key1_columns:
#             if song1_key == int(col[3:]):
#                 mix_df[col][row_count] = 1 
#             else:
#                 mix_df[col][row_count] = 0
#         for col in key2_columns:
#             if song2_key == int(col[3:]):
#                 mix_df[col][row_count] = 1 
#             else:
#                 mix_df[col][row_count] = 0

        if song2_tempo > tempo_low_bound and song2_tempo < tempo_high_bound:
            if song1_mode == 1: # Key is Major
                if song1_key == song2_key and song1_mode != song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song2_key == (song1_key - 1) and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song1_key == min_song_key and song2_key == max_song_key and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                else:
                    mix_df["y"][row_count] = 0
            if song1_mode == 0: #Key is Minor
                if song2_key == (song1_key - 1) and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                elif song1_key == min_song_key and song2_key == max_song_key and song1_mode == song2_mode:
                    mix_df["y"][row_count] = 1 
                else:
                    mix_df["y"][row_count] = 0
        else:
            mix_df["y"][row_count] = 0 

        
        if(row_count % 100 == 0):
            prog = ((row_count + 1)/ rng) * 100
            print("### progress: {:.2f} % ({}/{})".format(prog, row_count+1, rng))
        row_count += 1

C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### progress: 0.00 % (1/250000)


C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### progress: 0.04 % (101/250000)
### progress: 0.08 % (201/250000)
### progress: 0.12 % (301/250000)
### progress: 0.16 % (401/250000)
### progress: 0.20 % (501/250000)


C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### progress: 0.24 % (601/250000)
### progress: 0.28 % (701/250000)
### progress: 0.32 % (801/250000)
### progress: 0.36 % (901/250000)
### progress: 0.40 % (1001/250000)
### progress: 0.44 % (1101/250000)
### progress: 0.48 % (1201/250000)
### progress: 0.52 % (1301/250000)
### progress: 0.56 % (1401/250000)
### progress: 0.60 % (1501/250000)


C:\Users\roike\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [282]:
#Display Data


mix_df[mix_df["y"] == 1].count()

name1      2380
artist1    2380
key1       2380
mode1      2380
tempo1     2380
name2      2380
artist2    2380
key2       2380
mode2      2380
tempo2     2380
y          2380
key11      2380
key12      2380
key13      2380
key14      2380
key15      2380
key16      2380
key17      2380
key18      2380
key19      2380
key110     2380
key111     2380
key112     2380
key21      2380
key22      2380
key23      2380
key24      2380
key25      2380
key26      2380
key27      2380
key28      2380
key29      2380
key210     2380
key211     2380
key212     2380
dtype: int64

In [264]:
out_file_name = "mix_df_boost.xlsx" #161 for 10000
out_file_name = os.path.join(DATA_DIR, out_file_name)
mix_df.to_excel(out_file_name,index=False)

In [273]:
out_file_name = "mix_df_drop.xlsx" #164 for 10000
out_file_name = os.path.join(DATA_DIR, out_file_name)
mix_df.to_excel(out_file_name,index=False)

In [283]:
out_file_name = "mix_df_boost_250000.xlsx" #161 for 10000
out_file_name = os.path.join(DATA_DIR, out_file_name)
mix_df.to_excel(out_file_name,index=False)

In [ ]:
out_file_name = "mix_df_drop_250000.xlsx" #164 for 10000
out_file_name = os.path.join(DATA_DIR, out_file_name)
mix_df.to_excel(out_file_name,index=False)

In [290]:
PICKLE_DIR = "DataPickles"
out_file_name = "mix_df_boost_250000.pickle"
out_pickle = os.path.join(PICKLE_DIR, out_file_name)
mix_df.to_pickle(out_pickle)

In [291]:
read_df = pd.read_pickle(out_pickle)
